In [1]:
from noworkflow.now.persistence.models import Evaluation, Activation
from noworkflow.now.models.dependency_querier import DependencyQuerier
from noworkflow.now.models.dependency_querier.node_context import NodeContext
from noworkflow.now.models.dependency_querier.querier_options import QuerierOptions
from noworkflow.now.persistence.models import Trial
from copy import copy

import numpy as np
from pprint import pprint

In [2]:
import numpy as np

# Helper functions
def get_activation(activation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Activation((trial_id, activation_id))

def get_evaluation(evaluation_id, trial_id = None):
    if not trial_id:
        trial_id =  __noworkflow__.trial_id
    return Evaluation((trial_id, evaluation_id))

def get_act_evals(activation_id):
    activation = get_activation(activation_id)
    return activation.evaluations    

def print_act_forward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependent_dependencies):
        pprint.pprint(i.to_dict())
        
def print_act_backward(activation_id=1):
    activation = get_activation(activation_id)
    
    for i in list(activation.dependency_dependencies):
        pprint.pprint(i.to_dict())

def print_act_evals(activation_id=1):
    activation = get_activation(activation_id)

    for item in activation.evaluations:
        pprint.pprint(item.to_dict())
        


### Test case

Here starts my (only) test case. The flow goes through cells defining module, mae and rmse, variable definitions, variable manipulation and _var_final_ being our target variable, being dependent of its previous cells.


In [3]:
def module(num): 
    if num >= 0:
        return int(num)
    else:
        return int(-1*num)

def mae(x, y):
    return np.mean(np.abs((x - y)))

def rmse(x, y):
    return np.sqrt(np.mean((x - y)**2))

def stress_func(m, n):
    temp = m * n
    x = 10
    y = 100
    
    temp = np.random.rand()*temp
    
    final = np.max([int(x+y), int(temp)])
    
    return final

In [4]:
var_a = 10

In [5]:
var_b = -100

In [6]:
x = 50
y = 30

In [7]:
rme_err = mae(x, var_b)

In [8]:
var_c = stress_func(x, y)

In [9]:
var_d = var_c * rme_err

In [10]:
var_e = mae(var_c, var_b)
x = var_e + 10

In [11]:
#k = module(rmse_err)
var_final = now_variable('var_final', rmse(var_c, x))

### New functions test

In [12]:
evals = get_evaluation(131)

In [13]:
evals

evaluation(9236ebae-a8e2-4b65-9222-02f7a9454868, 131, 6.446164974, 591, 124, '110.0', 144).

### Outputing the dependencies of a variable

print_precedents() trying to output a cleaner output from get_precedent() results

In [ ]:
class NotebookQuerierOptions(QuerierOptions):
    global body_function_def
    #global dep_dict
    dep_list = []
    
    def visit_arrow(self, context, neighbor):
                
        # keeping 
        if neighbor.evaluation.code_component.type == 'function_def':
            body_function_def.append(int(neighbor.evaluation.code_component.id))
       
        arrow_list = ('argument', '<.__class__>', 'item')
        type_list = ('add', 'literal', 'mult', 'div', 'param', 'sub', 'attribute', 'usub', 'function_def')
        
        context_code_comp = context.evaluation.code_component
        neighbor_code_comp = neighbor.evaluation.code_component
        
        if neighbor.arrow not in arrow_list:
            if context_code_comp.type not in type_list:
                if neighbor_code_comp.type not in type_list:
                    if not (neighbor.arrow == 'use' and context_code_comp.type == 'call'):
                        if (neighbor_code_comp.container_id != None):
                            if (neighbor_code_comp.container_id not in body_function_def) or self.level:
                                self.dep_list.append((str(context.evaluation.checkpoint), str(context.evaluation.id), context_code_comp.name, context.evaluation.repr))

    def public_print(self):
        global dep_dict
        
        dep_dict = {i[0] : i[1] for i in enumerate(self.dep_list)}
        return dep_dict
        
                                
body_function_def = []
dep_dict = {}

nbOptions = NotebookQuerierOptions(False)
querier = DependencyQuerier(options=nbOptions)
nodes_to_visit, visited, found = querier.navigate_dependencies([evals])  



IndexError: pop from empty list

In [ ]:
nbOptions.public_print()